In [1]:
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [2]:
dataset = tfds.load('yelp_polarity_reviews')

train_dataset = dataset['train']
test_dataset = dataset['test']

train_texts = []
train_labels = []

for x in train_dataset:
    train_texts.append(x['text'].numpy().decode())
    train_labels.append([int(x['label'].numpy())])

test_texts = []
test_labels = []

for x in test_dataset:
    test_texts.append(x['text'].numpy().decode())
    test_labels.append([int(x['label'].numpy())])

Metal device set to: Apple M1


2023-06-01 13:35:13.407781: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [3]:
vocab_size = 10_000
maxlen = 140

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=maxlen)
test_padded = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=maxlen)

train_padded, train_labels = np.array(train_padded), np.array(train_labels)
test_padded, test_labels = np.array(test_padded), np.array(test_labels)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [4]:
model.fit(train_padded, train_labels, epochs=1, validation_data=(test_padded, test_labels))

17500/17500 [==============================] - 845s 48ms/step - loss: 0.1823 - accuracy: 0.9287 - val_loss: 0.1411 - val_accuracy: 0.9475


In [13]:
labels = [1, 1, 0, 1]
reviews = ['I absolutely like this restaurant',
         'I love the salad!',
         'I did not like the service',
         'The university is great']

In [14]:
tokenized_reviews = tokenizer.texts_to_sequences(reviews)
reviews_preprocessed = tf.keras.preprocessing.sequence.pad_sequences(tokenized_reviews, maxlen=maxlen)

In [15]:
predictions = model.predict(reviews_preprocessed)

1/1 [==============================] - 0s 183ms/step


In [16]:
for i in range(len(reviews)):
    print(f'Review: {reviews[i]}')
    print(f'Prediction: {predictions[i][0]}')
    print(f'Label: {labels[i]}')
    print()

Review: I absolutely like this restaurant
Prediction: 0.8920900225639343
Label: 1

Review: I love the salad!
Prediction: 0.9826284646987915
Label: 1

Review: I did not like the service
Prediction: 0.2976422607898712
Label: 0

Review: The university is great
Prediction: 0.9514614939689636
Label: 1

